# Реализация нахождения уравнений движения с помощью Урванений Татаринова для платформы с одним колесом


In [39]:
import sympy as sp
import cloudpickle as pickle

from copy import copy
from sympy import Derivative, Symbol, IndexedBase, Eq, Idx, Sum, Function, Matrix, Add
from sympy import diff, symbols, solve, simplify, poly, pprint, factor, integrate, lambdify
from sympy import cos, sin, pi
from abc import ABC, abstractmethod

In [40]:
def create_fs(self, fs):
    """ not good """
    # nu1, nu2, nu3 = symbols('nu1, nu2, nu3')
    return fs.subs({
        _nu1: Function('nu1_')(t),
        _nu2: Function('nu2_')(t),
        _nu3: Function('nu3_')(t)
    })

def create_ss(self, fs):
    """ not good """
    # nu1, nu2, nu3 = symbols('nu1, nu2, nu3')
    return fs.subs({
        Function('nu1_')(t): _nu1,
        Function('nu2_')(t): _nu2,
        Function('nu3_')(t): _nu3
    })

def save(data, name):
    with open(f'{name}.pickle', 'wb') as f:
        pickle.dump(copy(data), f)
        
def load(name):
    with open(f'{name}.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

def MatrixElDerivative(mat):
    return Matrix([x.diff(t) for x in mat])

### Уравнения Татаринова
$$
\frac{d}{dt} \frac{\partial L*}{\partial \omega_{\alpha}} +
\{P_{\alpha}, L*\} = 
\{P_{\alpha}, \sum_{\mu}{\omega_{\mu} P_{\mu}} \} +
\sum_{i}{Q_i \frac{\partial v_i}{ \partial \omega_{\alpha}}}
$$

### Скобка Пуассона
$$
\{f, g\} = \sum_{i=1}^{N} \left( \frac{\partial f}{\partial q_{i}} \frac{\partial g}{\partial p_{i}} - \frac{\partial f}{\partial p_i} \frac{\partial g}{\partial q_i}\right).
$$

### Функции $ P_{\alpha} $
$$
\sum_{\alpha}{P_{\alpha} \omega_{\alpha}} = \sum_i{p_iv_i}
$$

### Функции $v_i$
$$
v_i = \dot{q}
$$

In [41]:
class MechanicalSystem():
    
    def __init__(self):
        pass
    
    def set_q(self, q):
        self.q = q
    
    def set_p(self, p):
        self.p = p
    
    def poisson_bracket(self, F, G): # надо сделать, чтобы использовалось IndexedBase
        """ To evaluate an unevaluated derivative, use the doit method. """
        res = 0
        for i in range(3):
            res += diff(F, self.q[i])*diff(G, self.p[i])
            res -= diff(F, self.p[i])*diff(G, self.q[i])
        return res
    
    @staticmethod
    def sub_Eq(equation, Eq):
        return equation.subs(Eq.args[0], Eq.args[1])
    
    @classmethod
    def sub_Eqs(cls, equation, Eqs):
        sub_equation = equation
        for Eq in Eqs:
            sub_equation = cls.sub_Eq(sub_equation, Eq)
        return sub_equation 
    
    @staticmethod
    def left_part_Eqs(Eqs):
        return [Eq.args[0] for Eq in Eqs]
    
    @staticmethod
    def right_part_Eqs(Eqs):
        return [Eq.args[1] for Eq in Eqs]
    
    def set_constraints(self, constraints):
        """ Setting up constraints """
        self.constraints = constraints
        
    def sub_constraints(self, func):
        """ Substitutes constraints in the equation
        Private special case:
            L -> L* 
        """
        sub_dict = {conn.args[0]: conn.args[1] for conn in self.constraints}
        return func.subs(sub_dict)
    
    def sub_constraints_to_list(self, equations):
        return [self.sub_constraints(equation) for equation in equations]
    
    @staticmethod
    def diff_hack(equation, by):
        """ eq -> eq.subs(Derivative -> temp_variable) -> eq.diff(temp_variable) ->
            -> eq.subs(temp_variable -> Derivative)
        """
        tmp_by = Symbol('tmp_by')
        tmp_eq = equation.subs({by: tmp_by})
        tmp_eq = tmp_eq.diff(tmp_by)
        return tmp_eq.subs({tmp_by: by})
        

In [42]:
class TatarinovSystem(MechanicalSystem):
    
    def __init__(self, N=3, debug=True):
        super().__init__()
        self.N = N
        self.tatarinov_equations = [None]*N
        self.debug = True
        
    def set_omega_equations(self, omegas, equations):
        omega_equations = [Eq(omega, equation) for omega, equation in zip(omegas, equations)]
        self.omega_equations = omega_equations
        
    def set_v_equations(self, vs, equations): # vs - не лучшее название
        v_equations = [Eq(v, equation) for v, equation in zip(vs, equations)]
        self.v_equations = v_equations

    def set_P(self):
        k, mu = symbols('k, mu', cls=Idx)
        left = Sum(P[k]*om[k], (k, 1, self.N)).doit()
        right = Sum(p[i]*v[i], (k, 1, self.N))
        Eq(left, right)
    
    def set_L(self, L):
        self.L = L
        
    def set_L_star(self, L_star):
        self.L_star = simplify(L_star)
    
    def set_F(self, F):
        self.F = F
    
    def create_r(self):
        r = {}
        r['s'] = x*e['x'] + y*e['y']
        r['p'] = r['s'] + xi*e['xi'] + eta*e['eta']
        self.r = r
        return self.r

    def set_Qs(self, Qs):
        self.Qs = Qs
    
    def create_P(self):
        equations_for_P = Eq(
            Sum(P[k]*_omega[k], (k, 1, S.N)),
            Sum(p[k]*_v[k], (k, 1, S.N)))
        print(equations_for_P)
        # subs omega_i and v_i
        sub_equations_for_P = TatarinovSystem.sub_Eqs(
            equations_for_P.doit(), # important
            self.omega_equations + self.v_equations) # for Ex: omega[1] -> nu1), v[1] -> x', ....
        # subs_constraints
        sub_conn_equations_for_P = self.sub_constraints(sub_equations_for_P)
        print(sub_conn_equations_for_P)
        # to equate the coefficients
        left_coeffs = poly(sub_conn_equations_for_P.args[0], TatarinovSystem.right_part_Eqs(self.omega_equations)).coeffs()
        right_coeffs = poly(sub_conn_equations_for_P.args[1], TatarinovSystem.right_part_Eqs(self.omega_equations)).coeffs()
        final_equations_for_P = [Eq(left, right) for left, right in zip(left_coeffs, right_coeffs)]
        self.Ps = final_equations_for_P
        return final_equations_for_P
    
    def create_Q(self, F, r_p):
        self.Q = [F.dot(r_p.diff(x)) for x in self.q]
        
    def set_Q(self, Q):
        self.Q = Q
    
    def Q_dw_by_dv(self, i):
        Q_dw_by_dv = lambda i,a: self.Q[i]*Derivative(self.right_part_Eqs(S.sub_constraints_to_list(S.v_equations))[i],
                                                      self.right_part_Eqs(self.omega_equations)[a]).doit()
        Q_dw_by_dv_sum = lambda a: Q_dw_by_dv(0,a) + Q_dw_by_dv(1,a) + Q_dw_by_dv(2,a) + Q_dw_by_dv(3,a) + Q_dw_by_dv(4,a)
        return Q_dw_by_dv_sum(i)
    
    def tatarinov_equation__depricated(self, i):
        left, right = 0, 0;
        left += diff(diff(L, omega[k]), t)
        left += poisson_bracket(P[k], L)
        right += poisson_bracket(P[k], Sum(omega[k]*P[k], (k, 1, self.N)))
        right += Sum(Q[l]*diff(v[l], omega[k]), (l, 1, self.N))
        Eq(left, right)
        
    def create_bracket_sum(self):
        """ not good """
        # return  _nu1*(p[1]*cos(alpha) + p[2]*sin(alpha)) + _nu2*(-p[1]*sin(alpha) + p[2]*cos(alpha)) + _nu3*p[3]
        bracket_sum = [x*y for x,y in zip(S.right_part_Eqs(S.Ps), S.right_part_Eqs(S.omega_equations))]
        res = 0
        for x in range(3):
            res += bracket_sum[x]
        return res
    
    def solve_tatarinov_equations(self):
        te = [self.tatarinov_equations[i] for x in range(3)]
        sol = solve(te, [Derivative(self.create_fs(x), t) for x in S.right_part_Eqs(S.omega_equations)])
        return sol

    def tatarinov_equation(self, i):
        left, right = 0,0
        left = self.L_star.args[1]
#         left = self.create_fs(left.diff(self.right_part_Eqs(self.omega_equations)[i])).diff(t)
        left = left.diff(self.right_part_Eqs(self.omega_equations)[i])
        left = simplify(left)
        left = left.diff(t)
        left = simplify(self.sub_constraints(left))
        tmp = self.poisson_bracket(self.Ps[i].args[1], self.L_star.args[1])
        left += simplify(self.sub_constraints(tmp))
#         left = self.create_fs(left)
        if self.debug:
            display(left)
        
        right += self.poisson_bracket(self.Ps[i].args[1], self.create_bracket_sum())
        right += simplify(self.Q_dw_by_dv(i))
#         right = self.create_fs(right)
        if self.debug:
            display(right)
        
        ps = lambda i: self.sub_constraints(self.diff_hack(self.L.args[1], self.right_part_Eqs(self.v_equations)[i]))
        self.tatarinov_equations[i] = Eq(left, right).subs({p[1]: ps(0), p[2]: ps(1), p[3]: ps(2)})
        
#         self.tatarinov_equations[i] = self.create_fs(simplify(self.tatarinov_equations[i]))
        self.tatarinov_equations[i] = simplify(self.tatarinov_equations[i])
        
        return self.tatarinov_equations[i]

    def display_tatarinov_equations(self):
        for eq in self.tatarinov_equations:
            display(eq)

##  Импорт платформы

In [43]:
%load_ext autoreload
%autoreload 2

from variables import *
from structure import *
from functions import *

from latex import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Псевдоскорости $nu_1$, $nu_2$
#### Координаты $\theta_1$, $\psi_1$, $\alpha$
#### Связи $\psi_1(\nu_1,\nu_2), \theta_1(\nu_1,\nu_2)$

#### Скорости

In [44]:
omega['platform']= lambda i: Derivative(alpha,t)*e['z']
omega['fork']    = lambda i: omega['platform'](i) + Derivative(theta[i],t)*e['z']
omega['wheel']   = lambda i: omega['fork'](i) + Derivative(psi[i],t)*n_wheel(i)

In [45]:
v['S'] = euler(S, P)
v['P'] = euler(P, C)
v['C'] = euler(C, D)
v['D'] = lambda i: Matrix([0,0,0]) # проскальзывания нет

In [46]:
v['S'](0)

Matrix([
[ -d*(sin(alpha(t))*cos(theta0(t)) + sin(theta0(t))*cos(alpha(t)))*(Derivative(alpha(t), t) + Derivative(theta0(t), t)) - r*(-sin(alpha(t))*sin(theta0(t)) + cos(alpha(t))*cos(theta0(t)))*Derivative(psi0(t), t) - (sin(beta0)*cos(alpha(t)) + sin(alpha(t))*cos(beta0))*Derivative(alpha(t), t)],
[d*(-sin(alpha(t))*sin(theta0(t)) + cos(alpha(t))*cos(theta0(t)))*(Derivative(alpha(t), t) + Derivative(theta0(t), t)) + r*(-sin(alpha(t))*cos(theta0(t)) - sin(theta0(t))*cos(alpha(t)))*Derivative(psi0(t), t) + (-sin(beta0)*sin(alpha(t)) + cos(beta0)*cos(alpha(t)))*Derivative(alpha(t), t)],
[                                                                                                                                                                                                                                                                                               0]])

#### Угловые скорости

In [47]:
omega['platform']= lambda i: Derivative(alpha,t)*e['z']
omega['fork']    = lambda i: omega['platform'](i) + Derivative(theta[i],t)*e['z']
omega['wheel']   = lambda i: omega['fork'](i) + Derivative(psi[i],t)*n_wheel(i)

In [48]:
eq['nu1'] = lambda i: scalar(v['S'](i), e['xi'])
eq['nu2'] = lambda i: scalar(v['S'](i), e['eta'])

#### Псевдоскорости

In [49]:
simplify(Matrix([Eq(nu[1], eq['nu1'](0)),
                 Eq(nu[2], eq['nu2'](0))]))

Matrix([
[Eq(nu1(t), -d*sin(theta0(t))*Derivative(alpha(t), t) - d*sin(theta0(t))*Derivative(theta0(t), t) - r*cos(theta0(t))*Derivative(psi0(t), t) - sin(beta0)*Derivative(alpha(t), t))],
[ Eq(nu2(t), d*cos(theta0(t))*Derivative(alpha(t), t) + d*cos(theta0(t))*Derivative(theta0(t), t) - r*sin(theta0(t))*Derivative(psi0(t), t) + cos(beta0)*Derivative(alpha(t), t))]])

In [50]:
eq['f(nu1,nu2)'] = lambda i: solve(
                      [Eq(eq['nu1'](i), nu[1]), Eq(eq['nu2'](i), nu[2])],
                      [Derivative(psi[i],t), Derivative(theta[i],t)],
                      dict=True)[0]; # возвращает словарь с выражениями для diff(psi) и diff(theta)

In [51]:
Eq(Derivative(alpha), nu[3])

Eq(Derivative(alpha(t), t), nu3(t))

####  Связи

In [52]:
eq['diff(psi)_nu']   = lambda i: eq['f(nu1,nu2)'](i)[Derivative(psi[i],t)]
eq['diff(theta)_nu'] = lambda i: eq['f(nu1,nu2)'](i)[Derivative(theta[i],t)]

In [53]:
Matrix([Eq(Derivative(psi[0], t), eq['diff(psi)_nu'](0)),
        Eq(Derivative(theta[0], t), eq['diff(theta)_nu'](0))])

Matrix([
[                              Eq(Derivative(psi0(t), t), -(nu1(t)*cos(theta0(t)) + nu2(t)*sin(theta0(t)) + sin(beta0 - theta0(t))*Derivative(alpha(t), t))/r)],
[Eq(Derivative(theta0(t), t), (-d*Derivative(alpha(t), t) - nu1(t)*sin(theta0(t)) + nu2(t)*cos(theta0(t)) - cos(beta0 - theta0(t))*Derivative(alpha(t), t))/d)]])

In [54]:
# L, m, x, y, t, alpha = symbols('L, m, x, y, t, alpha')
# _nu1, _nu2, _nu3, _nu4, _nu5 = symbols('nu1_, nu2_, nu3_, nu4_, nu5_')
# W, T = symbols('W, T')
# s = symbols('s')
P = IndexedBase('P')
p = IndexedBase('p')
# p = IndexedBase('p')
_omega = IndexedBase('omega')
_v = IndexedBase('v')
# Q = IndexedBase('Q')
k = symbols('k', cls=Idx)
# J = IndexedBase('J')

# xi, eta = symbols('xi, eta')
# e = {}
# e['x'] = Matrix([1,0])
# e['y'] = Matrix([0,1])
# e['xi'] = Matrix([cos(alpha), sin(alpha)])
# e['eta'] = Matrix([-sin(alpha), cos(alpha)])

In [55]:
# A, C, c = symbols('A, C, c')

# psi, theta, phi = symbols('psi, theta, phi') # v

# g = symbols('g')

$$ L = T - V $$
$$ T = T_{pl} + T_{fo} + T_{wh} $$

$$ T_{pl} = \frac{m_{pl} v_S^2}{2} + \frac{J_{pl} \omega_{pl}^2}{2} $$
$$ T_{fo} = 0$$
$$ T_{wh} = \frac{m_{wh} v_S^2}{2} + \frac{J_{wh} \omega_{wh}^2}{2} $$

#### Platform

In [56]:
m['platform']

m1

In [57]:
Derivative(x, t)**2 + Derivative(y, t)**2

Derivative(x(t), t)**2 + Derivative(y(t), t)**2

In [58]:
J['platform'] = eye(3,3)*a

In [59]:
omega['platform'](0)

Matrix([
[                      0],
[                      0],
[Derivative(alpha(t), t)]])

#### Wheel

In [60]:
m['wheel']

m2

In [61]:
Matrix([x,y,0]) + d*e_wheel(0)

Matrix([
[d*(-sin(alpha(t))*sin(theta0(t)) + cos(alpha(t))*cos(theta0(t))) + x(t)],
[ d*(sin(alpha(t))*cos(theta0(t)) + sin(theta0(t))*cos(alpha(t))) + y(t)],
[                                                                      0]])

In [62]:
J['wheel'] = Matrix([[b,0,0],[0,c,0],[0,0,b]])

#### В одинаковых базисах нужно записывать угловую скорость и момент инерции !

In [63]:
omega['wheel_in'] = lambda i: psi[i].diff(t)*e['y'] + (theta[i].diff(t) + alpha.diff(t))*e['z']

In [64]:
omega['wheel_in'](0)

Matrix([
[                                                 0],
[                            Derivative(psi0(t), t)],
[Derivative(alpha(t), t) + Derivative(theta0(t), t)]])

### Моменты сил
####  (platform) $ \vec{M} = -W_0*\vec{e}_z $ 
####  (fork) $ \vec{M} = W_0*\vec{e}_z - T_0*\vec{n}_0 $ 
####  (wheel) $ \vec{M} = T_0*\vec{n}_0 $ 

In [65]:
eq['nu1(x,y)'] = Eq(nu[1], Derivative(x,t)*cos(alpha) + Derivative(y,t)*sin(alpha))
eq['nu2(x,y)'] = Eq(nu[2], -Derivative(x,t)*sin(alpha) + Derivative(y,t)*cos(alpha))

In [66]:
eq['diff_x(nu1,nu2)'], eq['diff_y(nu1,nu2)'] = linsolve([eq['nu1(x,y)'], eq['nu2(x,y)']], (Derivative(x, t), Derivative(y, t))).args[0]

In [67]:
def subs_xy(obj):
    substed = obj.subs({nu[1]: eq['nu1(x,y)'],
                        nu[2]: eq['nu2(x,y)']})
    return substed

### L

In [71]:
L = Symbol('L')

In [72]:
pl = m['platform']/2*(Derivative(x, t)**2 + Derivative(y, t)**2)
pl += (J['platform']*omega['platform'](0)).dot(omega['platform'](0))/2

wh = m['wheel']/2*(MatrixElDerivative(Matrix([x,y,0]) + d*e_wheel(0))).dot(MatrixElDerivative(Matrix([x,y,0]) + d*e_wheel(0)))
wh += (J['wheel']*omega['wheel_in'](0)).dot(omega['wheel_in'](0))/2

In [73]:
L = Eq(L, pl + wh)
L

Eq(L, a*Derivative(alpha(t), t)**2/2 + b*(Derivative(alpha(t), t) + Derivative(theta0(t), t))**2/2 + c*Derivative(psi0(t), t)**2/2 + m1*(Derivative(x(t), t)**2 + Derivative(y(t), t)**2)/2 + m2*((d*(-sin(alpha(t))*sin(theta0(t))*Derivative(alpha(t), t) - sin(alpha(t))*sin(theta0(t))*Derivative(theta0(t), t) + cos(alpha(t))*cos(theta0(t))*Derivative(alpha(t), t) + cos(alpha(t))*cos(theta0(t))*Derivative(theta0(t), t)) + Derivative(y(t), t))**2 + (d*(-sin(alpha(t))*cos(theta0(t))*Derivative(alpha(t), t) - sin(alpha(t))*cos(theta0(t))*Derivative(theta0(t), t) - sin(theta0(t))*cos(alpha(t))*Derivative(alpha(t), t) - sin(theta0(t))*cos(alpha(t))*Derivative(theta0(t), t)) + Derivative(x(t), t))**2)/2)

In [74]:
%%time
sL = simplify(L)

CPU times: user 7.61 s, sys: 0 ns, total: 7.61 s
Wall time: 7.61 s


In [75]:
sL

Eq(L, a*Derivative(alpha(t), t)**2/2 + b*(Derivative(alpha(t), t) + Derivative(theta0(t), t))**2/2 + c*Derivative(psi0(t), t)**2/2 + m1*(Derivative(x(t), t)**2 + Derivative(y(t), t)**2)/2 + m2*((d*(Derivative(alpha(t), t) + Derivative(theta0(t), t))*sin(alpha(t) + theta0(t)) - Derivative(x(t), t))**2 + (d*(Derivative(alpha(t), t) + Derivative(theta0(t), t))*cos(alpha(t) + theta0(t)) + Derivative(y(t), t))**2)/2)

### Определим связи 

In [98]:
conn1 = Eq(Derivative(x, t), eq['diff_x(nu1,nu2)'])
conn2 = Eq(Derivative(y, t), eq['diff_y(nu1,nu2)'])
conn3 = Eq(Derivative(alpha, t), nu[3])
conn4 = Eq(Derivative(theta[0], t), simplify(eq['diff(theta)_nu'](0))).subs({Derivative(alpha, t): nu[3]})
conn5 = Eq(Derivative(psi[0], t), simplify(eq['diff(psi)_nu'](0))).subs({Derivative(alpha, t): nu[3]})

In [99]:
for conn in [conn1, conn2, conn3, conn4, conn5]:
    display(conn)

Eq(Derivative(x(t), t), nu1(t)*cos(alpha(t)) - nu2(t)*sin(alpha(t)))

Eq(Derivative(y(t), t), nu1(t)*sin(alpha(t)) + nu2(t)*cos(alpha(t)))

Eq(Derivative(alpha(t), t), nu3(t))

Eq(Derivative(theta0(t), t), (-d*nu3(t) - nu1(t)*sin(theta0(t)) + nu2(t)*cos(theta0(t)) - nu3(t)*cos(beta0 - theta0(t)))/d)

Eq(Derivative(psi0(t), t), -(nu1(t)*cos(theta0(t)) + nu2(t)*sin(theta0(t)) + nu3(t)*sin(beta0 - theta0(t)))/r)

## Заполним что нужно (связи, уравнения для $\omega_i$, $v_i$, $F$)

### Создадим объект для механической системы с уравнениями Татаринова 

In [100]:
S = TatarinovSystem()

In [101]:
S.set_L(sL)

In [103]:
S.set_constraints([conn1, conn2, conn3, conn4, conn5])
S.constraints

[Eq(Derivative(x(t), t), nu1(t)*cos(alpha(t)) - nu2(t)*sin(alpha(t))),
 Eq(Derivative(y(t), t), nu1(t)*sin(alpha(t)) + nu2(t)*cos(alpha(t))),
 Eq(Derivative(alpha(t), t), nu3(t)),
 Eq(Derivative(theta0(t), t), (-d*nu3(t) - nu1(t)*sin(theta0(t)) + nu2(t)*cos(theta0(t)) - nu3(t)*cos(beta0 - theta0(t)))/d),
 Eq(Derivative(psi0(t), t), -(nu1(t)*cos(theta0(t)) + nu2(t)*sin(theta0(t)) + nu3(t)*sin(beta0 - theta0(t)))/r)]

In [104]:
%%time
sL_star = S.set_L_star(S.sub_constraints(sL))

CPU times: user 1min 44s, sys: 15.8 ms, total: 1min 44s
Wall time: 1min 44s


In [105]:
S.set_p([p[1], p[2], p[3]]) # это скорее внутренняя переменная и её не надо задавать here 
S.set_q([x, y, alpha, theta[0], psi[0]])

In [106]:
S.set_omega_equations(omegas=[_omega[1], _omega[2], _omega[3]],
                      equations=[nu[1], nu[2], nu[3]])
S.omega_equations

[Eq(omega[1], nu1(t)), Eq(omega[2], nu2(t)), Eq(omega[3], nu3(t))]

In [107]:
S.set_v_equations(vs=[_v[1], _v[2], _v[3], _v[4], _v[5]],
                  equations=[Derivative(x, t), Derivative(y, t), Derivative(alpha, t), Derivative(theta[0], t), Derivative(psi[0], t)])
S.v_equations

[Eq(v[1], Derivative(x(t), t)),
 Eq(v[2], Derivative(y(t), t)),
 Eq(v[3], Derivative(alpha(t), t)),
 Eq(v[4], Derivative(theta0(t), t)),
 Eq(v[5], Derivative(psi0(t), t))]

### Создадим $P_{\alpha}$

In [108]:
Ps = S.create_P()
Ps

Eq(Sum(P[k]*omega[k], (k, 1, 3)), Sum(p[k]*v[k], (k, 1, 3)))
Eq(nu1(t)*P[1] + nu2(t)*P[2] + nu3(t)*P[3], (nu1(t)*sin(alpha(t)) + nu2(t)*cos(alpha(t)))*p[2] + (nu1(t)*cos(alpha(t)) - nu2(t)*sin(alpha(t)))*p[1] + nu3(t)*p[3])


[Eq(P[1], sin(alpha(t))*p[2] + cos(alpha(t))*p[1]),
 Eq(P[2], -sin(alpha(t))*p[1] + cos(alpha(t))*p[2]),
 Eq(P[3], p[3])]

### Создадим $Q_i$

In [109]:
# F = Matrix([0,0,0])
# S.set_F(F)

# S.create_Q(S.F, S.create_r()['p'])
# S.Q

S.set_Q([0,0,0,W[1],T[1]])

In [115]:
for i in range(3):
    display(S.Q_dw_by_dv(i))

-T1(t)*cos(theta0(t))/r - W1(t)*sin(theta0(t))/d

-T1(t)*sin(theta0(t))/r + W1(t)*cos(theta0(t))/d

-T1(t)*sin(beta0 - theta0(t))/r + (-d - cos(beta0 - theta0(t)))*W1(t)/d

### Получаем уравнения

##### Проверим что функция работает верно

In [ ]:
i = 0

In [ ]:
left, right = 0,0

In [ ]:
left = S.L_star.args[1]

In [175]:
tmp = S.right_part_Eqs(S.omega_equations)[i]
tmp

nu1(t)

In [176]:
tmp = left.diff(tmp)
tmp

(2*b*r**2*(nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)) + nu3(t)*cos(beta0 - theta0(t)))*sin(theta0(t)) + 2*c*d**2*(nu1(t)*cos(theta0(t)) + nu2(t)*sin(theta0(t)) + nu3(t)*sin(beta0 - theta0(t)))*cos(theta0(t)) + d**2*r**2*(2*m1*nu1(t) + m2*((2*sin(alpha(t) + theta0(t))*sin(theta0(t)) + 2*cos(alpha(t)))*(nu1(t)*sin(alpha(t) + theta0(t))*sin(theta0(t)) + nu1(t)*cos(alpha(t)) - 2*nu2(t)*sin(alpha(t)) - nu2(t)*sin(theta0(t))*cos(alpha(t) + theta0(t)) + nu3(t)*sin(alpha(t) + theta0(t))*cos(beta0 - theta0(t))) + (-2*sin(alpha(t)) + 2*sin(theta0(t))*cos(alpha(t) + theta0(t)))*(-nu1(t)*sin(alpha(t)) + nu1(t)*sin(theta0(t))*cos(alpha(t) + theta0(t)) + nu2(t)*sin(alpha(t) + theta0(t))*sin(theta0(t)) - 2*nu2(t)*cos(alpha(t)) + nu3(t)*cos(beta0 - theta0(t))*cos(alpha(t) + theta0(t))))))/(2*d**2*r**2)

In [177]:
%%time
tmp = simplify(tmp.diff(t))

CPU times: user 1min 5s, sys: 0 ns, total: 1min 5s
Wall time: 1min 5s


In [178]:
tmp

b*nu1(t)*sin(2*theta0(t))*Derivative(theta0(t), t)/d**2 - b*nu2(t)*cos(2*theta0(t))*Derivative(theta0(t), t)/d**2 + b*nu3(t)*cos(beta0 - 2*theta0(t))*Derivative(theta0(t), t)/d**2 + b*sin(beta0)*Derivative(nu3(t), t)/(2*d**2) - b*sin(beta0 - 2*theta0(t))*Derivative(nu3(t), t)/(2*d**2) - b*sin(2*theta0(t))*Derivative(nu2(t), t)/(2*d**2) - b*cos(2*theta0(t))*Derivative(nu1(t), t)/(2*d**2) + b*Derivative(nu1(t), t)/(2*d**2) - c*nu1(t)*sin(2*theta0(t))*Derivative(theta0(t), t)/r**2 + c*nu2(t)*cos(2*theta0(t))*Derivative(theta0(t), t)/r**2 - c*nu3(t)*cos(beta0 - 2*theta0(t))*Derivative(theta0(t), t)/r**2 + c*sin(beta0)*Derivative(nu3(t), t)/(2*r**2) + c*sin(beta0 - 2*theta0(t))*Derivative(nu3(t), t)/(2*r**2) + c*sin(2*theta0(t))*Derivative(nu2(t), t)/(2*r**2) + c*cos(2*theta0(t))*Derivative(nu1(t), t)/(2*r**2) + c*Derivative(nu1(t), t)/(2*r**2) + m1*Derivative(nu1(t), t) + 3*m2*nu1(t)*sin(2*theta0(t))*Derivative(theta0(t), t) - 3*m2*nu2(t)*cos(2*theta0(t))*Derivative(theta0(t), t) + 2*m2*nu

In [113]:
left = tmp

In [144]:
S.L

Eq(L, a*Derivative(alpha(t), t)**2/2 + b*(Derivative(alpha(t), t) + Derivative(theta0(t), t))**2/2 + c*Derivative(psi0(t), t)**2/2 + m1*(Derivative(x(t), t)**2 + Derivative(y(t), t)**2)/2 + m2*((d*(Derivative(alpha(t), t) + Derivative(theta0(t), t))*sin(alpha(t) + theta0(t)) - Derivative(x(t), t))**2 + (d*(Derivative(alpha(t), t) + Derivative(theta0(t), t))*cos(alpha(t) + theta0(t)) + Derivative(y(t), t))**2)/2)

In [150]:
simplify(S.L_star.args[1].subs({alpha: 8}) - S.L_star.args[1]) # !

0

In [151]:
S.Ps[i].args[1]

sin(alpha(t))*p[2] + cos(alpha(t))*p[1]

In [153]:
simplify(S.L_star.args[1])

(b*r**2*(nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)) + nu3(t)*cos(beta0 - theta0(t)))**2 + c*d**2*(nu1(t)*cos(theta0(t)) + nu2(t)*sin(theta0(t)) + nu3(t)*sin(beta0 - theta0(t)))**2 + d**2*r**2*(a*nu3(t)**2 + m1*(nu1(t)**2 + nu2(t)**2) + m2*((-nu1(t)*sin(alpha(t)) + nu1(t)*sin(theta0(t))*cos(alpha(t) + theta0(t)) + nu2(t)*sin(alpha(t) + theta0(t))*sin(theta0(t)) - 2*nu2(t)*cos(alpha(t)) + nu3(t)*cos(beta0 - theta0(t))*cos(alpha(t) + theta0(t)))**2 + (nu1(t)*sin(alpha(t) + theta0(t))*sin(theta0(t)) + nu1(t)*cos(alpha(t)) - 2*nu2(t)*sin(alpha(t)) - nu2(t)*sin(theta0(t))*cos(alpha(t) + theta0(t)) + nu3(t)*sin(alpha(t) + theta0(t))*cos(beta0 - theta0(t)))**2)))/(2*d**2*r**2)

In [114]:
tmp = S.poisson_bracket(S.Ps[i].args[1], S.L_star.args[1])

In [115]:
left += tmp

In [166]:
simplify(S.sub_constraints(left))

(b*d*r**2*(sin(theta0(t))*Derivative(nu1(t), t) + cos(beta0 - theta0(t))*Derivative(nu3(t), t) - cos(theta0(t))*Derivative(nu2(t), t))*sin(theta0(t)) + b*r**2*(-nu1(t)*sin(2*theta0(t)) + nu2(t)*cos(2*theta0(t)) - nu3(t)*cos(beta0 - 2*theta0(t)))*(d*nu3(t) + nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)) + nu3(t)*cos(beta0 - theta0(t))) + c*d**3*(sin(beta0)*Derivative(nu3(t), t) + sin(beta0 - 2*theta0(t))*Derivative(nu3(t), t) + sin(2*theta0(t))*Derivative(nu2(t), t) + cos(2*theta0(t))*Derivative(nu1(t), t) + Derivative(nu1(t), t))/2 + c*d**2*(nu1(t)*sin(2*theta0(t)) - nu2(t)*cos(2*theta0(t)) + nu3(t)*cos(beta0 - 2*theta0(t)))*(d*nu3(t) + nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)) + nu3(t)*cos(beta0 - theta0(t))) + d**3*r**2*(2*m1*Derivative(nu1(t), t) + 2*m2*sin(beta0)*Derivative(nu3(t), t) - 2*m2*sin(beta0 - 2*theta0(t))*Derivative(nu3(t), t) - 3*m2*sin(2*theta0(t))*Derivative(nu2(t), t) - 3*m2*cos(2*theta0(t))*Derivative(nu1(t), t) + 5*m2*Derivative(nu1(t), t))/2 + d**2*m2*r**2*(-3

In [50]:
right

0

In [51]:
S.Ps[i].args[1]

sin(alpha(t))*p[2] + cos(alpha(t))*p[1]

In [52]:
S.create_bracket_sum()

(-sin(alpha(t))*p[1] + cos(alpha(t))*p[2])*nu2(t) + (sin(alpha(t))*p[2] + cos(alpha(t))*p[1])*nu1(t) + nu3(t)*p[3]

In [53]:
S.poisson_bracket(S.Ps[i].args[1], S.create_bracket_sum())

(-sin(alpha(t))*p[1] + cos(alpha(t))*p[2])*nu3(t)

In [54]:
right += S.poisson_bracket(S.Ps[i].args[1], S.create_bracket_sum())

In [56]:
right

(-sin(alpha(t))*p[1] + cos(alpha(t))*p[2])*nu3(t)

In [122]:
S.Q_dw_by_dv(i)

0

In [123]:
right += simplify(S.Q_dw_by_dv(i))

In [124]:
S.L.args[1]

a*Derivative(alpha(t), t)**2/2 + b*(Derivative(alpha(t), t) + Derivative(theta0(t), t))**2/2 + c*Derivative(psi0(t), t)**2/2 + m1*(Derivative(x(t), t)**2 + Derivative(y(t), t)**2)/2 + m2*((d*(Derivative(alpha(t), t) + Derivative(theta0(t), t))*sin(alpha(t) + theta0(t)) - Derivative(x(t), t))**2 + (d*(Derivative(alpha(t), t) + Derivative(theta0(t), t))*cos(alpha(t) + theta0(t)) + Derivative(y(t), t))**2)/2

In [125]:
S.right_part_Eqs(S.v_equations)[i]

Derivative(x(t), t)

In [126]:
S.sub_constraints(S.diff_hack(S.L.args[1], S.right_part_Eqs(S.v_equations)[i]))

m1*(nu1(t)*cos(alpha(t)) - nu2(t)*sin(alpha(t))) + m2*(-2*d*(nu3(t) + (-d*nu3(t) - nu1(t)*sin(theta0(t)) + nu2(t)*cos(theta0(t)) - nu3(t)*cos(beta0 - theta0(t)))/d)*sin(alpha(t) + theta0(t)) + 2*nu1(t)*cos(alpha(t)) - 2*nu2(t)*sin(alpha(t)))/2

In [127]:
simplify(S.sub_constraints(S.diff_hack(S.L.args[1], S.right_part_Eqs(S.v_equations)[i])))

m1*(nu1(t)*cos(alpha(t)) - nu2(t)*sin(alpha(t))) - m2*((-nu1(t)*sin(theta0(t)) + nu2(t)*cos(theta0(t)) - nu3(t)*cos(beta0 - theta0(t)))*sin(alpha(t) + theta0(t)) - nu1(t)*cos(alpha(t)) + nu2(t)*sin(alpha(t)))

In [128]:
ps = lambda i: simplify(S.sub_constraints(S.diff_hack(S.L.args[1], S.right_part_Eqs(S.v_equations)[i])))

In [129]:
ps(0)

m1*(nu1(t)*cos(alpha(t)) - nu2(t)*sin(alpha(t))) - m2*((-nu1(t)*sin(theta0(t)) + nu2(t)*cos(theta0(t)) - nu3(t)*cos(beta0 - theta0(t)))*sin(alpha(t) + theta0(t)) - nu1(t)*cos(alpha(t)) + nu2(t)*sin(alpha(t)))

In [130]:
ps(1)

m1*(nu1(t)*sin(alpha(t)) + nu2(t)*cos(alpha(t))) + m2*((-nu1(t)*sin(theta0(t)) + nu2(t)*cos(theta0(t)) - nu3(t)*cos(beta0 - theta0(t)))*cos(alpha(t) + theta0(t)) + nu1(t)*sin(alpha(t)) + nu2(t)*cos(alpha(t)))

In [131]:
ps(2)

(b*(-nu1(t)*sin(theta0(t)) + nu2(t)*cos(theta0(t)) - nu3(t)*cos(beta0 - theta0(t))) + d*(a*nu3(t) + d*m2*(-2*nu1(t)*sin(theta0(t)) + 2*nu2(t)*cos(theta0(t)) - nu3(t)*cos(beta0 - theta0(t)))))/d

In [132]:
ps(3)

(b*(-nu1(t)*sin(theta0(t)) + nu2(t)*cos(theta0(t)) - nu3(t)*cos(beta0 - theta0(t))) + d**2*m2*(-2*nu1(t)*sin(theta0(t)) + 2*nu2(t)*cos(theta0(t)) - nu3(t)*cos(beta0 - theta0(t))))/d

In [133]:
S.v_equations

[Eq(v[1], Derivative(x(t), t)),
 Eq(v[2], Derivative(y(t), t)),
 Eq(v[3], Derivative(alpha(t), t)),
 Eq(v[4], Derivative(theta0(t), t)),
 Eq(v[5], Derivative(psi0(t), t))]

In [134]:
Eq(left, right)

Eq(b*nu1(t)*sin(2*theta0(t))*Derivative(theta0(t), t)/d**2 - b*nu2(t)*cos(2*theta0(t))*Derivative(theta0(t), t)/d**2 + b*nu3(t)*cos(beta0 - 2*theta0(t))*Derivative(theta0(t), t)/d**2 + b*sin(beta0)*Derivative(nu3(t), t)/(2*d**2) - b*sin(beta0 - 2*theta0(t))*Derivative(nu3(t), t)/(2*d**2) - b*sin(2*theta0(t))*Derivative(nu2(t), t)/(2*d**2) - b*cos(2*theta0(t))*Derivative(nu1(t), t)/(2*d**2) + b*Derivative(nu1(t), t)/(2*d**2) - c*nu1(t)*sin(2*theta0(t))*Derivative(theta0(t), t)/r**2 + c*nu2(t)*cos(2*theta0(t))*Derivative(theta0(t), t)/r**2 - c*nu3(t)*cos(beta0 - 2*theta0(t))*Derivative(theta0(t), t)/r**2 + c*sin(beta0)*Derivative(nu3(t), t)/(2*r**2) + c*sin(beta0 - 2*theta0(t))*Derivative(nu3(t), t)/(2*r**2) + c*sin(2*theta0(t))*Derivative(nu2(t), t)/(2*r**2) + c*cos(2*theta0(t))*Derivative(nu1(t), t)/(2*r**2) + c*Derivative(nu1(t), t)/(2*r**2) + m1*Derivative(nu1(t), t) + 3*m2*nu1(t)*sin(2*theta0(t))*Derivative(theta0(t), t) - 3*m2*nu2(t)*cos(2*theta0(t))*Derivative(theta0(t), t) + 2*m2

In [88]:
Eq(left, right).subs({p[1]: ps(0), p[2]: ps(1), p[3]: ps(2)})

Eq(0, ((m1*(nu1(t)*sin(alpha(t)) + nu2(t)*cos(alpha(t))) + m2*((d*nu3(t) - d*Derivative(alpha(t), t) - nu1(t)*sin(theta0(t)) + nu2(t)*cos(theta0(t)) - cos(beta0 - theta0(t))*Derivative(alpha(t), t))*cos(alpha(t) + theta0(t)) + nu1(t)*sin(alpha(t)) + nu2(t)*cos(alpha(t))))*cos(alpha(t)) - (m1*(nu1(t)*cos(alpha(t)) - nu2(t)*sin(alpha(t))) - m2*((d*nu3(t) - d*Derivative(alpha(t), t) - nu1(t)*sin(theta0(t)) + nu2(t)*cos(theta0(t)) - cos(beta0 - theta0(t))*Derivative(alpha(t), t))*sin(alpha(t) + theta0(t)) - nu1(t)*cos(alpha(t)) + nu2(t)*sin(alpha(t))))*sin(alpha(t)))*nu3(t))

# ...

## Equations

In [117]:
%%time 
S.tatarinov_equation(0)

(-b*r**2*(nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)) + nu3(t)*cos(beta0 - theta0(t)))*(d*nu3(t) + nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)) + nu3(t)*cos(beta0 - theta0(t)))*cos(theta0(t)) - b*r**2*(d*nu1(t)*nu3(t)*cos(theta0(t)) + d*nu2(t)*nu3(t)*sin(theta0(t)) + d*nu3(t)**2*sin(beta0 - theta0(t)) - d*sin(theta0(t))*Derivative(nu1(t), t) - d*cos(beta0 - theta0(t))*Derivative(nu3(t), t) + d*cos(theta0(t))*Derivative(nu2(t), t) + nu1(t)**2*sin(2*theta0(t))/2 - nu1(t)*nu2(t)*cos(2*theta0(t)) + nu1(t)*nu3(t)*cos(beta0 - 2*theta0(t)) - nu2(t)**2*sin(2*theta0(t))/2 - nu2(t)*nu3(t)*sin(beta0 - 2*theta0(t)) + nu3(t)**2*sin(2*beta0 - 2*theta0(t))/2)*sin(theta0(t)) + d**2*(2*c*(nu1(t)*cos(theta0(t)) + nu2(t)*sin(theta0(t)) + nu3(t)*sin(beta0 - theta0(t)))*(d*nu3(t) + nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)) + nu3(t)*cos(beta0 - theta0(t)))*sin(theta0(t)) + 2*c*(d*(sin(beta0 - theta0(t))*Derivative(nu3(t), t) + sin(theta0(t))*Derivative(nu2(t), t) + cos(theta0(t))*Derivative(nu1(t), t

(-sin(alpha(t))*p[1] + cos(alpha(t))*p[2])*nu3(t) - T1(t)*cos(theta0(t))/r - W1(t)*sin(theta0(t))/d

CPU times: user 7min 6s, sys: 47.9 ms, total: 7min 6s
Wall time: 7min 6s


Eq((d*r*(2*m1*nu2(t) - m2*nu1(t)*sin(2*theta0(t)) + m2*nu2(t)*cos(2*theta0(t)) + 3*m2*nu2(t) - m2*nu3(t)*cos(beta0) - m2*nu3(t)*cos(beta0 - 2*theta0(t)))*nu3(t)/2 - d*T1(t)*cos(theta0(t)) - r*W1(t)*sin(theta0(t)))/(d*r), (-b*r**2*(nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)) + nu3(t)*cos(beta0 - theta0(t)))*(d*nu3(t) + nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)) + nu3(t)*cos(beta0 - theta0(t)))*cos(theta0(t)) - b*r**2*(2*d*nu1(t)*nu3(t)*cos(theta0(t)) + 2*d*nu2(t)*nu3(t)*sin(theta0(t)) + 2*d*nu3(t)**2*sin(beta0 - theta0(t)) - 2*d*sin(theta0(t))*Derivative(nu1(t), t) - 2*d*cos(beta0 - theta0(t))*Derivative(nu3(t), t) + 2*d*cos(theta0(t))*Derivative(nu2(t), t) + nu1(t)**2*sin(2*theta0(t)) - 2*nu1(t)*nu2(t)*cos(2*theta0(t)) + 2*nu1(t)*nu3(t)*cos(beta0 - 2*theta0(t)) - nu2(t)**2*sin(2*theta0(t)) - 2*nu2(t)*nu3(t)*sin(beta0 - 2*theta0(t)) + nu3(t)**2*sin(2*beta0 - 2*theta0(t)))*sin(theta0(t))/2 + d**2*(2*c*(nu1(t)*cos(theta0(t)) + nu2(t)*sin(theta0(t)) + nu3(t)*sin(beta0 - theta0(t)))*(d

In [118]:
%%time
S.tatarinov_equation(1)

(-b*r**2*(nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)) + nu3(t)*cos(beta0 - theta0(t)))*(d*nu3(t) + nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)) + nu3(t)*cos(beta0 - theta0(t)))*sin(theta0(t)) - b*r**2*(-d*nu1(t)*nu3(t)*cos(theta0(t)) - d*nu2(t)*nu3(t)*sin(theta0(t)) - d*nu3(t)**2*sin(beta0 - theta0(t)) + d*sin(theta0(t))*Derivative(nu1(t), t) + d*cos(beta0 - theta0(t))*Derivative(nu3(t), t) - d*cos(theta0(t))*Derivative(nu2(t), t) - nu1(t)**2*sin(2*theta0(t))/2 + nu1(t)*nu2(t)*cos(2*theta0(t)) - nu1(t)*nu3(t)*cos(beta0 - 2*theta0(t)) + nu2(t)**2*sin(2*theta0(t))/2 + nu2(t)*nu3(t)*sin(beta0 - 2*theta0(t)) - nu3(t)**2*sin(2*beta0 - 2*theta0(t))/2)*cos(theta0(t)) + d**2*(-2*c*(nu1(t)*cos(theta0(t)) + nu2(t)*sin(theta0(t)) + nu3(t)*sin(beta0 - theta0(t)))*(d*nu3(t) + nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)) + nu3(t)*cos(beta0 - theta0(t)))*cos(theta0(t)) + 2*c*(d*(sin(beta0 - theta0(t))*Derivative(nu3(t), t) + sin(theta0(t))*Derivative(nu2(t), t) + cos(theta0(t))*Derivative(nu1(t),

(-sin(alpha(t))*p[2] - cos(alpha(t))*p[1])*nu3(t) - T1(t)*sin(theta0(t))/r + W1(t)*cos(theta0(t))/d

CPU times: user 5min 58s, sys: 52 ms, total: 5min 58s
Wall time: 5min 58s


Eq((-d*r*(2*m1*nu1(t) - m2*nu1(t)*cos(2*theta0(t)) + 3*m2*nu1(t) - m2*nu2(t)*sin(2*theta0(t)) + m2*nu3(t)*sin(beta0) - m2*nu3(t)*sin(beta0 - 2*theta0(t)))*nu3(t)/2 - d*T1(t)*sin(theta0(t)) + r*W1(t)*cos(theta0(t)))/(d*r), (-b*r**2*(nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)) + nu3(t)*cos(beta0 - theta0(t)))*(d*nu3(t) + nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)) + nu3(t)*cos(beta0 - theta0(t)))*sin(theta0(t)) + b*r**2*(2*d*nu1(t)*nu3(t)*cos(theta0(t)) + 2*d*nu2(t)*nu3(t)*sin(theta0(t)) + 2*d*nu3(t)**2*sin(beta0 - theta0(t)) - 2*d*sin(theta0(t))*Derivative(nu1(t), t) - 2*d*cos(beta0 - theta0(t))*Derivative(nu3(t), t) + 2*d*cos(theta0(t))*Derivative(nu2(t), t) + nu1(t)**2*sin(2*theta0(t)) - 2*nu1(t)*nu2(t)*cos(2*theta0(t)) + 2*nu1(t)*nu3(t)*cos(beta0 - 2*theta0(t)) - nu2(t)**2*sin(2*theta0(t)) - 2*nu2(t)*nu3(t)*sin(beta0 - 2*theta0(t)) + nu3(t)**2*sin(2*beta0 - 2*theta0(t)))*cos(theta0(t))/2 + d**2*(-2*c*(nu1(t)*cos(theta0(t)) + nu2(t)*sin(theta0(t)) + nu3(t)*sin(beta0 - theta0(t)))*

In [119]:
%%time
S.tatarinov_equation(2)

(-b*r**2*(nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)) + nu3(t)*cos(beta0 - theta0(t)))*(d*nu3(t) + nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)) + nu3(t)*cos(beta0 - theta0(t)))*sin(beta0 - theta0(t)) - b*r**2*(d*nu1(t)*nu3(t)*cos(theta0(t)) + d*nu2(t)*nu3(t)*sin(theta0(t)) + d*nu3(t)**2*sin(beta0 - theta0(t)) - d*sin(theta0(t))*Derivative(nu1(t), t) - d*cos(beta0 - theta0(t))*Derivative(nu3(t), t) + d*cos(theta0(t))*Derivative(nu2(t), t) + nu1(t)**2*sin(2*theta0(t))/2 - nu1(t)*nu2(t)*cos(2*theta0(t)) + nu1(t)*nu3(t)*cos(beta0 - 2*theta0(t)) - nu2(t)**2*sin(2*theta0(t))/2 - nu2(t)*nu3(t)*sin(beta0 - 2*theta0(t)) + nu3(t)**2*sin(2*beta0 - 2*theta0(t))/2)*cos(beta0 - theta0(t)) + d**2*(c*(nu1(t)*cos(theta0(t)) + nu2(t)*sin(theta0(t)) + nu3(t)*sin(beta0 - theta0(t)))*(d*nu3(t) + nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)) + nu3(t)*cos(beta0 - theta0(t)))*cos(beta0 - theta0(t)) + c*(d*(sin(beta0 - theta0(t))*Derivative(nu3(t), t) + sin(theta0(t))*Derivative(nu2(t), t) + cos(theta0(t))*

-(-sin(alpha(t))*p[1] + cos(alpha(t))*p[2])*nu1(t) - (-sin(alpha(t))*p[2] - cos(alpha(t))*p[1])*nu2(t) - W1(t) - T1(t)*sin(beta0 - theta0(t))/r - W1(t)*cos(beta0 - theta0(t))/d

CPU times: user 7min 2s, sys: 164 ms, total: 7min 3s
Wall time: 7min 3s


Eq(-m2*nu1(t)**2*sin(2*theta0(t))/2 + m2*nu1(t)*nu2(t)*cos(2*theta0(t)) - m2*nu1(t)*nu3(t)*cos(beta0)/2 - m2*nu1(t)*nu3(t)*cos(beta0 - 2*theta0(t))/2 + m2*nu2(t)**2*sin(2*theta0(t))/2 - m2*nu2(t)*nu3(t)*sin(beta0)/2 + m2*nu2(t)*nu3(t)*sin(beta0 - 2*theta0(t))/2 + W1(t) + T1(t)*sin(beta0 - theta0(t))/r + W1(t)*cos(beta0 - theta0(t))/d, -(-b*r**2*(nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)) + nu3(t)*cos(beta0 - theta0(t)))*(d*nu3(t) + nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)) + nu3(t)*cos(beta0 - theta0(t)))*sin(beta0 - theta0(t)) - b*r**2*(2*d*nu1(t)*nu3(t)*cos(theta0(t)) + 2*d*nu2(t)*nu3(t)*sin(theta0(t)) + 2*d*nu3(t)**2*sin(beta0 - theta0(t)) - 2*d*sin(theta0(t))*Derivative(nu1(t), t) - 2*d*cos(beta0 - theta0(t))*Derivative(nu3(t), t) + 2*d*cos(theta0(t))*Derivative(nu2(t), t) + nu1(t)**2*sin(2*theta0(t)) - 2*nu1(t)*nu2(t)*cos(2*theta0(t)) + 2*nu1(t)*nu3(t)*cos(beta0 - 2*theta0(t)) - nu2(t)**2*sin(2*theta0(t)) - 2*nu2(t)*nu3(t)*sin(beta0 - 2*theta0(t)) + nu3(t)**2*sin(2*beta0 - 

In [122]:
save(S.tatarinov_equations, 'eqs+TW')

## Solve & Load

In [123]:
# eqs = load('eqs')
eqs = load('eqs+TW')

In [134]:
def par_subs(eq):
    return eq.subs({
        beta[0]: 0,
#         d: 2,
        r: 1,
        b: 2,
        c: 1,
        m['platform']: 3,
        m['wheel']: 1
    })

In [120]:
# simplify(par_subs(eqs[0]).subs(
#     {Derivative(x, t):0 for x in S.right_part_Eqs(S.omega_equations)}
# ))

# par_subs(eqs[0])

### Display equations 

In [ ]:
# %%time
# for eq in S.tatarinov_equations:
#     display(simplify(eq))

In [ ]:
%%time
res = linsolve([par_subs(eq) for eq in eqs],
               [Derivative(x, t) for x in S.right_part_Eqs(S.omega_equations)])

In [127]:
# save(res, 'res+TW')

In [128]:
# res = load('res')
res = load('res+TW')

In [129]:
%%time
s_eq1 = simplify(res.args[0][0])

CPU times: user 38 s, sys: 28 ms, total: 38 s
Wall time: 38 s


In [130]:
%%time
s_eq2 = simplify(res.args[0][1])

CPU times: user 25.5 s, sys: 44 ms, total: 25.6 s
Wall time: 25.6 s


In [131]:
%%time
s_eq3 = simplify(res.args[0][2])

CPU times: user 6.91 s, sys: 20 ms, total: 6.93 s
Wall time: 6.93 s


In [133]:
alpha

alpha(t)

In [132]:
for left_eq, right_eq in zip([s_eq1, s_eq2, s_eq3], [Derivative(x, t) for x in S.right_part_Eqs(S.omega_equations)]):
    display(Eq(left_eq, right_eq))

Eq((-(10*sin(2*theta0(t)) - sin(4*theta0(t)))*(24*T1(t)*sin(theta0(t))**3 + 48*T1(t)*sin(theta0(t)) - 30*W1(t)*sin(theta0(t))**2 + 10*W1(t)*cos(theta0(t))**3 - 30*W1(t)*cos(theta0(t)) - 60*W1(t) - 25*nu1(t)**2*sin(theta0(t))**5 - 40*nu1(t)**2*sin(theta0(t))**3 - 30*nu1(t)**2*sin(theta0(t))*cos(theta0(t))**3 + 20*nu1(t)**2*sin(theta0(t)) + 45*nu1(t)**2*sin(2*theta0(t)) - 20*nu1(t)*nu2(t)*sin(theta0(t))**6*cos(theta0(t)) - 40*nu1(t)*nu2(t)*sin(theta0(t))**4*cos(theta0(t)) + 60*nu1(t)*nu2(t)*sin(theta0(t))**4 + 90*nu1(t)*nu2(t)*sin(theta0(t))**2 - 20*nu1(t)*nu2(t)*cos(theta0(t))**7 + 150*nu1(t)*nu2(t)*cos(theta0(t))**5 - 330*nu1(t)*nu2(t)*cos(theta0(t))**3 + 180*nu1(t)*nu2(t)*cos(theta0(t)) - 60*nu1(t)*nu2(t) + 16*nu1(t)*nu3(t)*sin(theta0(t))**6*cos(theta0(t)) + 32*nu1(t)*nu3(t)*sin(theta0(t))**4*cos(theta0(t)) - 70*nu1(t)*nu3(t)*sin(theta0(t))**4 - 130*nu1(t)*nu3(t)*sin(theta0(t))**2 + 16*nu1(t)*nu3(t)*cos(theta0(t))**7 - 108*nu1(t)*nu3(t)*cos(theta0(t))**5 + 214*nu1(t)*nu3(t)*cos(theta0

Eq(-(2*(2*sin(theta0(t))**2 - 5)*(24*T1(t)*sin(theta0(t))**3 + 48*T1(t)*sin(theta0(t)) - 30*W1(t)*sin(theta0(t))**2 + 10*W1(t)*cos(theta0(t))**3 - 30*W1(t)*cos(theta0(t)) - 60*W1(t) - 25*nu1(t)**2*sin(theta0(t))**5 - 40*nu1(t)**2*sin(theta0(t))**3 - 30*nu1(t)**2*sin(theta0(t))*cos(theta0(t))**3 + 20*nu1(t)**2*sin(theta0(t)) + 45*nu1(t)**2*sin(2*theta0(t)) - 20*nu1(t)*nu2(t)*sin(theta0(t))**6*cos(theta0(t)) - 40*nu1(t)*nu2(t)*sin(theta0(t))**4*cos(theta0(t)) + 60*nu1(t)*nu2(t)*sin(theta0(t))**4 + 90*nu1(t)*nu2(t)*sin(theta0(t))**2 - 20*nu1(t)*nu2(t)*cos(theta0(t))**7 + 150*nu1(t)*nu2(t)*cos(theta0(t))**5 - 330*nu1(t)*nu2(t)*cos(theta0(t))**3 + 180*nu1(t)*nu2(t)*cos(theta0(t)) - 60*nu1(t)*nu2(t) + 16*nu1(t)*nu3(t)*sin(theta0(t))**6*cos(theta0(t)) + 32*nu1(t)*nu3(t)*sin(theta0(t))**4*cos(theta0(t)) - 70*nu1(t)*nu3(t)*sin(theta0(t))**4 - 130*nu1(t)*nu3(t)*sin(theta0(t))**2 + 16*nu1(t)*nu3(t)*cos(theta0(t))**7 - 108*nu1(t)*nu3(t)*cos(theta0(t))**5 + 214*nu1(t)*nu3(t)*cos(theta0(t))**3 - 102

Eq((-25*(1 - cos(2*theta0(t)))**2*nu1(t)**2*sin(theta0(t)) + 50*(1 - cos(2*theta0(t)))**2*nu1(t)*nu2(t)*cos(theta0(t)) + 60*(1 - cos(2*theta0(t)))**2*nu1(t)*nu2(t) - 28*(1 - cos(2*theta0(t)))**2*nu1(t)*nu3(t)*cos(theta0(t)) - 70*(1 - cos(2*theta0(t)))**2*nu1(t)*nu3(t) + 25*(1 - cos(2*theta0(t)))**2*nu2(t)**2*sin(theta0(t)) - 28*(1 - cos(2*theta0(t)))**2*nu2(t)*nu3(t)*sin(theta0(t)) - 8*(1 - cos(2*theta0(t)))**2*nu2(t)*nu3(t)*sin(2*theta0(t)) + 3*(1 - cos(2*theta0(t)))**2*nu3(t)**2*sin(theta0(t)) + 4*(1 - cos(2*theta0(t)))**2*nu3(t)**2*sin(2*theta0(t)) + 264*T1(t)*sin(theta0(t)) - 24*T1(t)*sin(3*theta0(t)) - 90*W1(t)*cos(theta0(t)) + 60*W1(t)*cos(2*theta0(t)) + 10*W1(t)*cos(3*theta0(t)) - 300*W1(t) - 40*nu1(t)**2*sin(theta0(t)) + 150*nu1(t)**2*sin(2*theta0(t)) + 40*nu1(t)**2*sin(3*theta0(t)) - 15*nu1(t)**2*sin(4*theta0(t)) + 10*nu1(t)*nu2(t)*cos(theta0(t)) - 180*nu1(t)*nu2(t)*cos(2*theta0(t)) - 90*nu1(t)*nu2(t)*cos(3*theta0(t)) - 60*nu1(t)*nu2(t) + 34*nu1(t)*nu3(t)*cos(theta0(t)) + 260*

In [116]:
for left_eq, right_eq in zip([s_eq1, s_eq2, s_eq3], [Derivative(x, t) for x in S.right_part_Eqs(S.omega_equations)]):
    display(Eq(left_eq, right_eq))

Eq(((10*sin(2*theta0(t)) - sin(4*theta0(t)))*(25*nu1(t)**2*sin(theta0(t))**5 + 40*nu1(t)**2*sin(theta0(t))**3 + 30*nu1(t)**2*sin(theta0(t))*cos(theta0(t))**3 - 20*nu1(t)**2*sin(theta0(t)) - 45*nu1(t)**2*sin(2*theta0(t)) + 20*nu1(t)*nu2(t)*sin(theta0(t))**6*cos(theta0(t)) + 40*nu1(t)*nu2(t)*sin(theta0(t))**4*cos(theta0(t)) - 60*nu1(t)*nu2(t)*sin(theta0(t))**4 - 90*nu1(t)*nu2(t)*sin(theta0(t))**2 + 20*nu1(t)*nu2(t)*cos(theta0(t))**7 - 150*nu1(t)*nu2(t)*cos(theta0(t))**5 + 330*nu1(t)*nu2(t)*cos(theta0(t))**3 - 180*nu1(t)*nu2(t)*cos(theta0(t)) + 60*nu1(t)*nu2(t) - 16*nu1(t)*nu3(t)*sin(theta0(t))**6*cos(theta0(t)) - 32*nu1(t)*nu3(t)*sin(theta0(t))**4*cos(theta0(t)) + 70*nu1(t)*nu3(t)*sin(theta0(t))**4 + 130*nu1(t)*nu3(t)*sin(theta0(t))**2 - 16*nu1(t)*nu3(t)*cos(theta0(t))**7 + 108*nu1(t)*nu3(t)*cos(theta0(t))**5 - 214*nu1(t)*nu3(t)*cos(theta0(t))**3 + 102*nu1(t)*nu3(t)*cos(theta0(t)) - 20*nu1(t)*nu3(t) - 25*nu2(t)**2*sin(theta0(t))**5 - 25*nu2(t)**2*sin(theta0(t))**3 - 30*nu2(t)**2*sin(thet

Eq((2*(2*sin(theta0(t))**2 - 5)*(25*nu1(t)**2*sin(theta0(t))**5 + 40*nu1(t)**2*sin(theta0(t))**3 + 30*nu1(t)**2*sin(theta0(t))*cos(theta0(t))**3 - 20*nu1(t)**2*sin(theta0(t)) - 45*nu1(t)**2*sin(2*theta0(t)) + 20*nu1(t)*nu2(t)*sin(theta0(t))**6*cos(theta0(t)) + 40*nu1(t)*nu2(t)*sin(theta0(t))**4*cos(theta0(t)) - 60*nu1(t)*nu2(t)*sin(theta0(t))**4 - 90*nu1(t)*nu2(t)*sin(theta0(t))**2 + 20*nu1(t)*nu2(t)*cos(theta0(t))**7 - 150*nu1(t)*nu2(t)*cos(theta0(t))**5 + 330*nu1(t)*nu2(t)*cos(theta0(t))**3 - 180*nu1(t)*nu2(t)*cos(theta0(t)) + 60*nu1(t)*nu2(t) - 16*nu1(t)*nu3(t)*sin(theta0(t))**6*cos(theta0(t)) - 32*nu1(t)*nu3(t)*sin(theta0(t))**4*cos(theta0(t)) + 70*nu1(t)*nu3(t)*sin(theta0(t))**4 + 130*nu1(t)*nu3(t)*sin(theta0(t))**2 - 16*nu1(t)*nu3(t)*cos(theta0(t))**7 + 108*nu1(t)*nu3(t)*cos(theta0(t))**5 - 214*nu1(t)*nu3(t)*cos(theta0(t))**3 + 102*nu1(t)*nu3(t)*cos(theta0(t)) - 20*nu1(t)*nu3(t) - 25*nu2(t)**2*sin(theta0(t))**5 - 25*nu2(t)**2*sin(theta0(t))**3 - 30*nu2(t)**2*sin(theta0(t))*cos(th

Eq((-25*(1 - cos(2*theta0(t)))**2*nu1(t)**2*sin(theta0(t)) + 50*(1 - cos(2*theta0(t)))**2*nu1(t)*nu2(t)*cos(theta0(t)) + 60*(1 - cos(2*theta0(t)))**2*nu1(t)*nu2(t) - 28*(1 - cos(2*theta0(t)))**2*nu1(t)*nu3(t)*cos(theta0(t)) - 70*(1 - cos(2*theta0(t)))**2*nu1(t)*nu3(t) + 25*(1 - cos(2*theta0(t)))**2*nu2(t)**2*sin(theta0(t)) - 28*(1 - cos(2*theta0(t)))**2*nu2(t)*nu3(t)*sin(theta0(t)) - 8*(1 - cos(2*theta0(t)))**2*nu2(t)*nu3(t)*sin(2*theta0(t)) + 3*(1 - cos(2*theta0(t)))**2*nu3(t)**2*sin(theta0(t)) + 4*(1 - cos(2*theta0(t)))**2*nu3(t)**2*sin(2*theta0(t)) - 40*nu1(t)**2*sin(theta0(t)) + 150*nu1(t)**2*sin(2*theta0(t)) + 40*nu1(t)**2*sin(3*theta0(t)) - 15*nu1(t)**2*sin(4*theta0(t)) + 10*nu1(t)*nu2(t)*cos(theta0(t)) - 180*nu1(t)*nu2(t)*cos(2*theta0(t)) - 90*nu1(t)*nu2(t)*cos(3*theta0(t)) - 60*nu1(t)*nu2(t) + 34*nu1(t)*nu3(t)*cos(theta0(t)) + 260*nu1(t)*nu3(t)*cos(2*theta0(t)) + 46*nu1(t)*nu3(t)*cos(3*theta0(t)) - 180*nu1(t)*nu3(t) - 125*nu2(t)**2*sin(theta0(t)) - 150*nu2(t)**2*sin(2*theta0(t)

## Найдем  $W,T, \ddot{\alpha}$  как функции от времени 

In [ ]:
from copy import copy

for x in S.tatarinov_equations:
    display(x)

### Зададим $x,y, alpha$ как функции времени

In [ ]:
# Начальные условия и парметры
eq_x = 3*t
eq_y = t**2

params = dict(xi=0, eta=0)

t_end = 2*pi.evalf()

In [ ]:
def _ss(eq):
    nu1, nu2, nu3 = symbols('nu1, nu2, nu3')
    return eq.subs({
        nu1: Function('nu1')(t),
        nu2: Function('nu2')(t),
        nu3: Function('nu3')(t)
    })
    
def subs_funcs(eq):
    subs_dict = dict(x=eq_x, y=eq_y, alpha=Function('alpha')(t)) # , alpha=eq_alpha)
    if type(eq) == dict:
        d = {}
        for k,v in eq.items():
            d[_ss(k)] = _ss(v.subs(subs_dict))
        return d
    if type(eq) == list:
        l = []
        for x in eq:
            l += _ss(eq.subs(subs_dict))
        return l
    if type(eq) == Eq or type(eq) == Add:
        return _ss(eq.subs(subs_dict))

In [ ]:
S.constraints

In [ ]:
nus = solve(S.constraints, S.right_part_Eqs(S.omega_equations))
for k, v in nus.items():
    display(Eq(k, v))

In [ ]:
nust = subs_funcs(nus)
nust

In [ ]:
tet = copy(S.tatarinov_equations)
for x in tet:
    display(x)

In [ ]:
for i in range(3):
    tet[i] = tet[i].subs(nust).doit()
    display(tet[i])
#     display((tet[i].subs(nust)))
#     display((tet[i].subs(nust)).doit())
    

In [ ]:
sol = solve(tet, [T, W, Derivative(Function('alpha')(t), t, 2)])
for k,v in sol.items():
    display(Eq(k, v))

### Проинтегрируем
$$ {\ddot{\alpha}, \dot{\alpha}} \rightarrow \alpha $$

In [ ]:
_vars = [Function('alpha2')(t), Function('alpha')(t)]
_vars

In [ ]:
eqs = [Function('alpha2')(t), list(sol.values())[2]]
eqs

In [ ]:
_vars = [Symbol('alpha'),  Symbol('alpha2')]
eqs =   [Symbol('alpha2'), list(sol.values())[2].subs({Function('alpha')(t): Symbol('alpha')})]

print("Система уравнений на alpha")
for l, r in zip(_vars, eqs):
    display(Eq(l,r))

In [ ]:
# keyword can't be an expression
eqs[1] = eqs[1].subs({J[s]: 3})

_subs = dict(m=1, eta=2, xi=3)

In [ ]:
def f(_vars_values, time):
    return [lambdify(_vars, x.subs(_subs), 'numpy')(*_vars_values) for x in eqs]

In [ ]:
import numpy as np
from scipy.integrate import RK45, odeint

In [ ]:
f0 = [1, 0.1] # [alpha', alpha]
time = np.linspace(0., 4, 100)

_solution = odeint(f, f0, time)

In [ ]:
import matplotlib.pyplot as plt

_items = zip(['dot_alpha', 'alpha'], list(range(2)))

for name, col in _items:
    print(name)
    fig = plt.plot(time, _solution[:, col], label=name)
    plt.show()

## Подставим в $W, T$
$$ \{W(\alpha, t), T(\alpha, t)\} \rightarrow \{W(t), T(t)\}$$

In [ ]:
display(sol[W])
display(sol[T])

In [ ]:
W_t = lambdify([t, Function('alpha')(t)], sol[W].subs(_subs), 'numpy')
T_t = lambdify([t, Function('alpha')(t)], sol[T].subs(_subs), 'numpy')

In [ ]:
_solution[:,1].size == time.size 

In [ ]:
wl, tl = [], []

for x,y in zip(time, _solution[:,1]):
    wl.append(W_t(x, y))
    
for x,y in zip(time, _solution[:,1]):
    tl.append(T_t(x, y))

In [ ]:
fig = plt.plot(time, wl, label='W')
plt.show()

In [ ]:
fig = plt.plot(time, tl, label='T')
plt.show()

In [ ]:
al = _solution[:, 1]

In [ ]:
tmp_x = tl*np.sin(al)*(-1) + wl*np.cos(al)
tmp_y = tl*np.cos(al) + wl*np.sin(al)

In [ ]:
fig = plt.plot(tmp_x, label='~x"')
fig = plt.plot(tmp_y, label='~y"')
plt.show()

In [ ]:
F

## Проинтегрируем численно

In [ ]:
left = []; right = []

In [ ]:
left += S.left_part_Eqs(S.constraints)
right += S.right_part_Eqs(S.constraints)

In [ ]:
diff_omegas_t = [Derivative(S.create_fs(x)) for x in S.right_part_Eqs(S.omega_equations)]
constr = solve(eqs, diff_omegas_t)
constr

In [ ]:
for left_item, right_item in constr.items():
    left += [left_item]
    right += [right_item]

In [ ]:
_vars = (phi, psi, theta, nu1, nu2, nu3)

In [ ]:
right = [S.create_ss(x) for x in right]

In [ ]:
_subs = {A: 1, C: 2, g: 10, m: 1}

In [ ]:
left

In [ ]:
right

In [ ]:
def f(_vars_values, time):
    return [lambdify(_vars, x.subs(_subs), 'numpy')(*_vars_values) for x in right]

In [ ]:
f0 = [3,0.1,0.1,1,2,3]

In [ ]:
import numpy as np
from scipy.integrate import RK45, odeint

In [ ]:
_solution = odeint(f, f0, np.linspace(0., 4., 100))

In [ ]:
import matplotlib.pyplot as plt 

_items = zip(['phi', 'psi', 'theta', 'nu1', 'nu2', 'nu3'], list(range(6)))

for name, col in _items:
    print(name)
    fig = plt.plot(np.linspace(0,4,100), _solution[:, col], label=name)
    plt.show()

## Посмотрим на энергию

In [ ]:
H = A*(Derivative(theta,t)**2 + Derivative(psi,t)**2*sin(theta)**2)/2 \
    + C*(Derivative(phi, t) + Derivative(psi, t)*cos(theta))**2/2 \
    + m*g*cos(theta)
H

In [ ]:
H = S.sub_constraints(H)
H

In [ ]:
fH = lambdify(_vars, H.subs(_subs), 'numpy')

In [ ]:
H_values = np.array([fH(*x) for x in _solution])
max(H_values) - min(H_values)

In [ ]:
for x in S.constraints:
    display(x)

In [ ]:
L

# $$ x(t), y(t), \alpha(t)$$

1. зададим $x(t), y(t), \alpha(t)$
2. 

In [ ]:
S.